# Datenvorbereitung
Zunächst werden für alle gewünschten Einflussfaktoren die Daten beschafft und in ein CSV-Format mit lat/lon oder GeoJSON-Geoemtry gebracht, sodass zu allen Adressen die gewünschten Eigenschaften vorliegen.

In [ ]:
import pandas as pd
from shapely.geometry import box

from helper import load_geocsv, s, clean_index_cols

EPSG_4326 = "EPSG:4326"

# BBOX für Brandenburg an der Havel
CITY_BOUNDING_BOX = box(12.3120236786, 52.2938432979, 12.7562682548, 52.5594777244)

#  Stadtzentrum für Brandenburg an der Havel
CITY_CENTER = (52.4116351153561, 12.556331280534392)

## Adressen einlesen
Alle Adressen des Zielgebiets als CSV in einen GeoDataFrame einlesen.

In [ ]:
gdf = load_geocsv("out/adressen_mit_zentrum_routen.csv")

# Adressen ohne Geometrie entfernen
gdf = gdf[~gdf.geometry.isna()].copy()

### Nutzungsart als Merkmal ergänzen
One-Hot-Encoding für spätere Modellierung der Bebauungsdichte / Nutzungsart der Adresse.

In [ ]:
import geopandas as gpd

gdf_bebauung = gpd.read_file("data/Bebauungsdichte/2025_Bebauungsdichte.shp")
gdf_bebauung = gdf_bebauung.to_crs(EPSG_4326)

gdf = gpd.sjoin(
    gdf,                                    # Punkte
    gdf_bebauung[["nutzart", "geometry"]],  # Polygone + Nutzungsart
    how="left",
    predicate="within"                      # point-in-polygon
)

gdf["nutzart"] = gdf["nutzart"].fillna("Unbekannt")


In [ ]:
mapping = {
    "Wohnbaufläche": "Wohnen",
    "Sport-, Freizeit- und Erholungsfläche": "Gruen",
    "Fläche gemischter Nutzung": "Gemischt",
    "Industrie- und Gewerbefläche": "Gewerbe",
    "Straßenverkehr": "Verkehr",
    "Weg": "Verkehr",
    "Platz": "Verkehr",
    "Friedhof": "Gruen",
    "Wald": "Gruen",
    "Fläche besonderer funktionaler Prägung": "Sonstiges",
}

# Neues zusammengefasstes Merkmal hinzufügen
gdf["nutzklasse"] = gdf["nutzart"].map(mapping).fillna("Sonstiges")

# One-Hot-Encoding der Nutzungsklasse für numerische Verarbeitung
# neu erzeugen
gdf_onehot = pd.get_dummies(gdf["nutzklasse"], prefix="nutz", dtype=int)
gdf = gdf.join(gdf_onehot)

# Fläche berechnen (in Meter-CRS)
gdf_bebauung_m = gdf_bebauung.to_crs(32633)
gdf_bebauung["area_sqm"] = gdf_bebauung_m.area

# Klassifizierung NACH mapping
gdf_bebauung["nutzklasse"] = gdf_bebauung["nutzart"].map(mapping).fillna("Sonstiges")

# Filter große Flächen (Schwellwert flexibel)
MIN_AREA = 100000
gdf_large = gdf_bebauung[gdf_bebauung["area_sqm"] > MIN_AREA].copy()
gdf


In [ ]:
import folium
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Farben für jede nutzklasse
klassen = gdf_large["nutzklasse"].unique()
cmap = plt.colormaps["Set2"]
colors = {k: mcolors.to_hex(cmap(i / len(klassen))) for i, k in enumerate(klassen)}

# Karte initialisieren
m = folium.Map(location=CITY_CENTER, zoom_start=13, tiles="OpenStreetMap")

# Ein Layer pro Klasse anlegen
layer_map = {}
for k in klassen:
    layer = folium.FeatureGroup(name=f"Große {k}-Flächen (> {MIN_AREA} m²)", show=True)
    layer_map[k] = layer
    m.add_child(layer)

# Große Flächen einzeichnen
# Transformieren
gdf_large_4326 = gdf_large.to_crs(4326)

# Vereinfachung (wichtig für Performance!)
gdf_large_4326["geometry"] = gdf_large_4326.geometry.simplify(
    tolerance=0.0002,  # ~20 m
    preserve_topology=True
)


for _, row in gdf_large_4326.iterrows():
    kls = row["nutzklasse"]
    layer = layer_map[kls]

    folium.GeoJson(
        row.geometry,
        style_function=lambda x, k=kls: {
            'color': 'black',
            'weight': 1,
            'fillColor': colors[k],
            'fillOpacity': 0.6
        },
        tooltip=folium.Tooltip(
            f"<b>{row['bez']}</b><br>"
            f"{kls}<br>"
            f"Fläche: {row['area_sqm']:.0f} m²"
        )
    ).add_to(layer)

# LayerControl aktivieren
folium.LayerControl(collapsed=False).add_to(m)

# open map in browser
m

## Explorative Datenanalyse

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns           # nur für schönere Plots
from helper import load_geocsv, make_merge_addr

# Eindeutigen Merge-baren String aus Adresse erzeugen
gdf["Adresse_merge"] = gdf.apply(make_merge_addr, axis=1)

gdf = gdf.rename(columns={"geojson": "center_route",
                                    "distance_m" : "center_distance_m"})

# Ein paar unbenötigte Spalten entfernen
gdf = gdf.drop(columns=["index_right", "duration_s", "display_name", "type", "category", "Adresse_query"], errors="ignore")

# Histogramm der Distanzen zum Zentrum
plt.figure(figsize=(6,3))
sns.histplot(gdf["center_distance_m"], bins=60, kde=False)
plt.title("Verteilung der Fußentfernung zum Zentrum")
plt.xlabel("Distanz (Meter)")
plt.ylabel("Häufigkeit")
plt.show()
print(gdf.shape)
print(gdf.columns)

## Daten bereinigen

In [ ]:
print(gdf.shape)
# Dubletten aus Adressen entfernen
dups = gdf["Adresse_merge"].value_counts()
dups = dups[dups > 1]
print(f"{len(dups)} doppelte Adressen gefunden")
print(dups.head())

# Ersten Treffer behalten
gdf = gdf.sort_values("Adresse_merge").drop_duplicates("Adresse_merge", keep="first")
print(gdf.shape)


## Einzelhandel
Separates Skript ```einzelhandel-adressen.py``` ausfühen, um Datei "adressen_mit_einzelhandel.csv" zu erzeugen.
Durch fußläufiges Routing berechnete Faktoren:
- Anzahl von Einkaufsmöglichkeiten im Umkreis von 500 m
- Anzahl von Einkaufsmöglichkeiten im Umkreis von 800 m
- Geringste Distanz zum nächsten Einzelhandel

In [ ]:
import seaborn as sns, matplotlib.pyplot as plt

# Lade vorberechnete Einzelhandel-Faktoren
gdf_retail = load_geocsv("out/adressen_mit_einzelhandel_routen.csv")

# Merge mit Haupt-GDF
gdf_retail["Adresse_merge"] = gdf_retail.apply(make_merge_addr, axis=1)
gdf_retail = gdf_retail.sort_values("Adresse_merge").drop_duplicates("Adresse_merge", keep="first")

dups = gdf_retail["Adresse_merge"].value_counts()
print(dups[dups > 1])

einzelhandel_attribute = ["einzelhandel_route","einzelhandel_min_distance_m", "einzelhandel_500m_count", "einzelhandel_800m_count", "einzelhandel_1000m_count"]

gdf = pd.merge(
    gdf,
    gdf_retail[["Adresse_merge"] + einzelhandel_attribute],
    on="Adresse_merge",
    how="left",
    validate="one_to_many"
)
print(gdf.shape)
print(gdf[["Adresse_merge", "einzelhandel_min_distance_m"]].head())

print(gdf.shape)

# Speicher freigeben
del(gdf_retail)

In [ ]:
# Verteilung Distanz zum nächsten Markt
print(gdf.columns)
sns.histplot(gdf["einzelhandel_min_distance_m"].dropna(), bins=40, kde=False)
plt.title("Distanz zum nächsten Lebensmittel­markt")
plt.xlabel("Meter Fußweg"); plt.ylabel("Adressen")
plt.show()

# Scatter Zentralität vs. Nahversorgung
sns.scatterplot(x="center_distance_m", y="einzelhandel_min_distance_m", data=gdf, alpha=.3)
plt.xlabel("Distanz Zentrum (m)"); plt.ylabel("Distanz nächster Markt (m)")
plt.show()

## Lärmbelastung
Siehe ```laerm.ipynb``` zur Erzeugung, ansonsten einfach ```data/adressen_mit_laerm.csv``` verwenden und per Spalte 'geometry' verschneiden.

Definition Lärm-Index:
- NaN / leer: kein gemessener Straßenlärm
- 0: 55 - 59 dB
- 1: 60 - 64 dB
- 2: 65 - 69 dB
- 3: 70 - 74 dB
- 4: >= 75 db

In [ ]:
gdf_laerm_karte = load_geocsv("out/adressen_mit_laerm.csv")
print(gdf_laerm_karte.shape)
print(gdf.shape)

# Merge des Lärmindex mit Haupt-GDF
gdf_laerm_karte["Adresse_merge"] = gdf_laerm_karte.apply(make_merge_addr, axis=1)
gdf_laerm_karte["laerm_index_tag"] = gdf_laerm_karte["Laerm_index_tag"].fillna(-0.1) # lowercase

# Deduplicate, im Zweifel nimm den lautesten Wert
gdf_laerm_karte = (
    gdf_laerm_karte
    .sort_values("laerm_index_tag", ascending=False)
    .drop_duplicates("Adresse_merge", keep="first")
)

laerm_attribute = ["laerm_index_tag"]
gdf = pd.merge(
    gdf,
    gdf_laerm_karte[["Adresse_merge"] + laerm_attribute],
    on="Adresse_merge",
    how="left",
    validate="one_to_many"
)

#print(gdf.columns)
print(gdf.shape)

## Bildung
Weitere Faktoren
- Fußläufige Entfernung zur nächstgelegenen Kita
- Fußläufige Entfernung zur nächstgelegenen Schule
- Anzahl Kitas im Umkreis von 500, 800 und 1000 Metern
- Anzahl Schulen im Umkreis von 500, 800 und 1000 Metern

In [ ]:
import pandas as pd
import geopandas as gpd

# Daten als GeoDataFrame einlesen
gdf_kitas = load_geocsv("out/adressen_mit_kitas_routen.csv")
gdf_grundschulen = load_geocsv("out/adressen_mit_grundschulen_routen.csv")
print(gdf_kitas.shape)
print(gdf_grundschulen.shape)
print(gdf.shape)

# Eindeutige Adresse für den Merge generieren
gdf_kitas["Adresse_merge"] = gdf_kitas.apply(make_merge_addr, axis=1)
gdf_kitas = gdf_kitas.drop_duplicates("Adresse_merge")
gdf_grundschulen["Adresse_merge"] = gdf_grundschulen.apply(make_merge_addr, axis=1)
gdf_grundschulen = gdf_grundschulen.drop_duplicates("Adresse_merge")

# Rename kitas_geometry column to kitas_route in gfd_kitas


# Merge into main gdf
kitas_attribute = ["kitas_min_distance_m", "kitas_route", "kitas_count_within_500m", "kitas_count_within_800m", "kitas_count_within_1000m"]
gdf = pd.merge(
    gdf,
    gdf_kitas[["Adresse_merge"] + kitas_attribute],
    on="Adresse_merge",
    how="left",
    validate="one_to_many"
)

grundschulen_attribute = ["grundschulen_min_distance_m", "grundschulen_route", "grundschulen_count_within_500m", "grundschulen_count_within_800m", "grundschulen_count_within_1000m"]
gdf = pd.merge(
    gdf,
    gdf_grundschulen[["Adresse_merge"] + grundschulen_attribute],
    on="Adresse_merge",
    how="left",
    validate="one_to_many"
)
del(gdf_kitas, gdf_grundschulen)  # Speicher freigeben

print(gdf.columns)
print(gdf.shape)

## Medizinische Versorgung
Ein "medizinisches Zentrum" wurde definiert als eine Apotheke mit zwei Ärzten im Umkreis von 100 Metern. Diese Zentren werden separat in ```medizinische-zentren.py``` berechnet. Dazu wird die euklidische Distanz ("Luftlinie") zwischen Apotheken und umgebenden Ärzten berechnet.
Dadurch entstehen für den Datensatz folgende neue Attribute:
- Anzahl von medizinischen Zentren im Umkreis von 500 m
- Anzahl von medizinischen Zentren im Umkreis von 800 m
- Anzahl von medizinischen Zentren im Umkreis von 1000 m
- Fußläufige Distanz zum nächsten medizinischen Zentrum

Die vollständige Erklärung der Felder findet sich im Anhang.

Die fußläufige Distanz zum nächstgelegenen medizinischen Zentrum wird per ```routing.py``` für jede Adresse einzeln ermittelt und als Weg gespeichert (```adressen_mit_medzentren_routen```).

In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Zentren / Apothekenstandorte mit Klassifikation
df_centers = pd.read_csv("out/medzentren_geocoded.csv")

# Normiere Bool-Spalte
df_centers["is_med_center"] = (df_centers["is_med_center"].astype(str).str.lower().isin(["true", "1", "yes", "y"]))

# Einzel-Apotheken ausblenden?
# df_centers = df_centers[df_centers["is_med_center"] == True].copy()

def build_popup(row):
    lines = []

    if bool(row["is_med_center"]):
        lines.append(f"<b>Medizinisches Zentrum \"{str(row['Strassenname'])}\"</b><br>")

    # Name
    if pd.notna(row.get("Name_Apotheke")) and str(row["Name_Apotheke"]).strip() != "":
        lines.append(str(row["Name_Apotheke"]))

    # Ärztedichte im 100m Radius
    if "arzt_count_100m" in row:
        lines.append(f"<br><b>{int(row['arzt_count_100m'])}</b> Arzt-Praxen")

    return "<br>".join(lines)

def pick_icon(row):
    if bool(row["is_med_center"]):
        return folium.Icon(color="green", icon="plus-sign", prefix="glyphicon")
    else:
        return folium.Icon(color="gray", icon="staff-snake", prefix="fa")

m = folium.Map(location=CITY_CENTER, zoom_start=14)
cluster = MarkerCluster(name="Medizinische Zentren / Apotheken")
cluster.add_to(m)

# Marker setzen
for _, row in df_centers.iterrows():
    lat = row["lat"]
    lon = row["lon"]

    if pd.isna(lat) or pd.isna(lon):
        continue

    popup_html = build_popup(row)
    icon = pick_icon(row)

    folium.Marker(
        location=[lat, lon],
        popup=popup_html,
        tooltip=row.get("Strassenname", "Apotheke"),
        icon=icon,
    ).add_to(cluster)

folium.LayerControl().add_to(m)
m

In [ ]:
# Medizinische Felder in gdf übernehmen
# 1. Routing-Ergebnis für medizinische Versorgung laden
gdf_med = load_geocsv("out/adressen_mit_medzentren_routen.csv")
print("gdf_med shape:", gdf_med.shape)
print("gdf shape (vor Merge):", gdf.shape)

# 2. Merge-Key bauen (Adresse normalisieren)
gdf_med["Adresse_merge"] = gdf_med.apply(make_merge_addr, axis=1)
gdf_med = gdf_med.drop_duplicates("Adresse_merge")

# 3. Relevante Attributspalten aus der medizinischen Versorgung definieren
medzentren_attribute = [
    "medzentren_min_distance_m",
    "medzentren_route",
    "medzentren_count_within_500m",
    "medzentren_count_within_800m",
    "medzentren_count_within_1000m",
]

# 4. Merge in Haupt-GDF
gdf = pd.merge(
    gdf,
    gdf_med[["Adresse_merge"] + medzentren_attribute],
    on="Adresse_merge",
    how="left",
    validate="one_to_one",
)

print("gdf shape (nach Merge):", gdf.shape)

## ÖPNV-Qualität

Die Qualität des ÖPNV wird anhand der Fußläufigkeit zur nächsten Haltestelle und der Häufigkeit von Abfahrten (Headway) bewertet. Die Daten stammen vom Verkehrsverbund Berlin-Brandenburg (VBB), Lizenz: CC BY 4.0, [zu den Daten](https://unternehmen.vbb.de/digitale-services/datensaetze).

Vorausgesetzte Datensätze:
- ```data/GTFS/stops.txt```, der die Haltestellen mit ihren Geokoordinaten enthält.
- ```data/adressen_mit_haltestellen_routen.csv```, der die Routen zu den Haltestellen und Anzahl von Haltestellen im Radius 500m und 800m enthält.

Probleme mit Datenqualität:
- In der "2024_Haltestellen.csv" fehlen die Haltestellen "Libellenweg" und "Immenweg" (Linie B). Dadurch werden diese Haltestellen nicht berücksichtigt und Wege zur nächsten Haltestelle werden länger eingeschätzt.

In [ ]:
from shapely.geometry import Point

# Load GTFS stops data
stops = pd.read_csv("data/GTFS/stops.txt")
stops["geometry"] = stops.apply(lambda row: Point(row["stop_lon"], row["stop_lat"]), axis=1)
gdf_stops = gpd.GeoDataFrame(stops, geometry="geometry", crs=EPSG_4326)

# Convert to UTM for precise metric distance calculations
gdf = gdf.to_crs(epsg=32633)
gdf_stops = gdf_stops.to_crs(epsg=32633)

# Assign nearest stop to each address based on Point(X, Y) geometry
gdf["nearest_stop_id"] = gdf.geometry.apply(
    lambda pt: gdf_stops.loc[gdf_stops.distance(pt).idxmin(), "stop_id"]
)

# Read routes from adresses to stops and stop count from prepared CSV
gdf_haltestellen = load_geocsv("out/adressen_mit_haltestellen_routen.csv")
gdf_haltestellen["Adresse_merge"] = gdf_haltestellen.apply(make_merge_addr, axis=1)
gdf_haltestellen = gdf_haltestellen.drop_duplicates("Adresse_merge").copy()

print("gdf_haltestellen shape:", gdf_haltestellen.shape)
print("gdf shape (vor Merge):", gdf.shape)

# Merge haltestellen attributes into main GeoDataFrame
haltestellen_attribute = ["haltestellen_route", "haltestellen_min_distance_m", "haltestellen_count_within_500m", "haltestellen_count_within_800m"]
gdf = pd.merge(
    gdf,
    gdf_haltestellen[["Adresse_merge"] + haltestellen_attribute],
    on="Adresse_merge",
    how="left",
    validate="one_to_one"
)

print("gdf shape (nach Merge):", gdf.shape)

# Speicher freigeben
del gdf_haltestellen

### Berechnung der ÖPNV-Taktung

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from sklearn.cluster import DBSCAN

# -------------------------------------------------
# 1. GTFS laden
# -------------------------------------------------
stops = pd.read_csv("data/GTFS/stops.txt")
stop_times = pd.read_csv("data/GTFS/stop_times.txt", low_memory=False)
trips = pd.read_csv("data/GTFS/trips.txt")
calendar = pd.read_csv("data/GTFS/calendar.txt")  # behalten wir für evtl. spätere Filter
# optional:
# calendar_dates = pd.read_csv("data/GTFS/calendar_dates.txt")

# -------------------------------------------------
# 2. Zeitspalte -> Minuten ab Mitternacht
# -------------------------------------------------
def parse_time_to_minutes(t):
    if pd.isna(t):
        return None
    parts = str(t).split(":")
    if len(parts) == 2:
        h, m = parts
    elif len(parts) == 3:
        h, m, _s = parts
    else:
        return None
    try:
        h = int(h)
        m = int(m)
        return h * 60 + m
    except ValueError:
        return None

stop_times = stop_times.copy()
stop_times["minutes"] = stop_times["arrival_time"].apply(parse_time_to_minutes)
stop_times = stop_times.dropna(subset=["minutes"])

# -------------------------------------------------
# 3. Wir interessieren uns für HVZ-Fenster
#    Wir extrahieren NUR Stop-Zeiten, die überhaupt in diesen Fenstern liegen.
#    Damit behalten wir reale Pendlerfahrten selbst dann,
#    wenn calendar.monday == 0 gesagt hätte.
# -------------------------------------------------

HVZ_WINDOWS = [
    (360, 540),   # 06:00–09:00
    (960, 1140),  # 16:00–19:00
]

def in_any_window(mins, windows):
    for lo, hi in windows:
        if lo <= mins <= hi:
            return True
    return False

stop_times_hvz = stop_times[stop_times["minutes"].apply(lambda mm: in_any_window(mm, HVZ_WINDOWS))].copy()

# 3. Filter nur auf werktägliche Dienste
trips_filtered = trips.merge(calendar, on="service_id")
trips_filtered = trips_filtered[trips_filtered["monday"] == 1]

# -------------------------------------------------
# 4. stop_times_hvz ↔ trips (route_id etc.)
# -------------------------------------------------
trips["trip_id"] = trips["trip_id"].astype(str)

stop_times_hvz["trip_id"] = stop_times_hvz["trip_id"].astype(str)

stopdata = stop_times_hvz.merge(
    trips[["trip_id", "route_id", "service_id"]],
    on="trip_id",
    how="left"
)

# -------------------------------------------------
# 5. Haltestellengeometrie + Clusterbildung (DBSCAN)
# -------------------------------------------------
stops = stops.copy()
stops["stop_id"] = stops["stop_id"].astype(str)

gdf_stops_all = gpd.GeoDataFrame(
    stops,
    geometry=gpd.points_from_xy(stops["stop_lon"], stops["stop_lat"]),
    crs=EPSG_4326
)

gdf_stops_metric = gdf_stops_all.to_crs(epsg=25833).copy()
coords = np.vstack([
    gdf_stops_metric.geometry.x.values,
    gdf_stops_metric.geometry.y.values
]).T

db = DBSCAN(eps=100, min_samples=1).fit(coords)
gdf_stops_all["pt_cluster_id"] = db.labels_.astype(int)

stop_to_cluster = dict(zip(gdf_stops_all["stop_id"], gdf_stops_all["pt_cluster_id"]))

# -------------------------------------------------
# 6. Cluster-ID an stopdata hängen
# -------------------------------------------------
stopdata["stop_id"] = stopdata["stop_id"].astype(str)
stopdata["pt_cluster_id"] = stopdata["stop_id"].map(stop_to_cluster)

# -------------------------------------------------
# 7. Headway je Cluster berechnen
#    jetzt auf Basis ALLER HVZ-Fahrten, die tatsächlich vorkommen,
#    statt nur calendar.monday==1
# -------------------------------------------------
def compute_headway_for_window(df, lo, hi, group_col="pt_cluster_id", time_col="minutes"):
    result = {}
    for clus, group in df.groupby(group_col):
        if pd.isna(clus):
            continue
        # nur Zeiten im Fenster
        times = sorted([t for t in group[time_col] if lo <= t <= hi])
        if len(times) < 2:
            continue
        diffs = [b - a for a, b in zip(times, times[1:])]
        result[clus] = sum(diffs) / len(diffs)
    return result

headway_morning = compute_headway_for_window(stopdata, 360, 540)
headway_evening = compute_headway_for_window(stopdata, 960, 1140)

df_hm = (
    pd.DataFrame.from_dict(headway_morning, orient="index", columns=["headway_morning"])
    .reset_index()
    .rename(columns={"index": "pt_cluster_id"})
)
df_he = (
    pd.DataFrame.from_dict(headway_evening, orient="index", columns=["headway_evening"])
    .reset_index()
    .rename(columns={"index": "pt_cluster_id"})
)

df_headways = df_hm.merge(df_he, on="pt_cluster_id", how="outer", validate="one_to_one")

# -------------------------------------------------
# 8. Merge in gdf
# -------------------------------------------------
gdf["nearest_stop_id"] = gdf["nearest_stop_id"].astype(str)
gdf["pt_cluster_id"] = gdf["nearest_stop_id"].map(stop_to_cluster)

# weg mit alten Spalten falls rerun
for col in [
    "headway_morning", "headway_evening", "headway_avg",
    "headway_morning_score_fixed", "headway_evening_score_fixed",
    "headway_avg_score_fixed",
]:
    if col in gdf.columns:
        gdf = gdf.drop(columns=[col])

gdf = gdf.merge(
    df_headways,
    on="pt_cluster_id",
    how="left",
    validate="many_to_one"
)

# -------------------------------------------------
# 9. Kennzahlen
# -------------------------------------------------
for col in ["headway_morning", "headway_evening"]:
    gdf[col] = pd.to_numeric(gdf[col], errors="coerce")

gdf["headway_avg"] = gdf[["headway_morning", "headway_evening"]].mean(axis=1)

fixed_min, fixed_max = 5, 60
def scoreify(series):
    return 1 - ((series - fixed_min) / (fixed_max - fixed_min)).clip(lower=0, upper=1)

gdf["headway_morning_score_fixed"] = scoreify(gdf["headway_morning"])
gdf["headway_evening_score_fixed"] = scoreify(gdf["headway_evening"])
gdf["headway_avg_score_fixed"]     = scoreify(gdf["headway_avg"])

# -------------------------------------------------
# 10. Debug neu
# -------------------------------------------------
no_headway_mask = gdf["headway_avg"].isna()

print("Adressen ohne Headway_avg:", int(no_headway_mask.sum()))

na_clusters = (
    gdf.loc[no_headway_mask, "pt_cluster_id"]
    .value_counts()
)
print("Problem-Cluster:", len(na_clusters))
print(na_clusters.head(20))

# sicherstellen, dass wir wirklich noch Stops ohne Window-Fahrten haben
clusters_with_headway = set(df_headways["pt_cluster_id"].tolist())
clusters_without = [cid for cid in gdf["pt_cluster_id"].dropna().unique()
                    if cid not in clusters_with_headway]
print("Cluster ohne Eintrag in df_headways:", len(clusters_without))
print(clusters_without[:20])

# Spot-check: nimm den größten Problem-Cluster
if len(clusters_without) > 0:
    test_cluster = clusters_without[0]
    sample_times = stopdata.loc[stopdata["pt_cluster_id"] == test_cluster, "minutes"]
    print("Beispiel-Cluster", test_cluster, "HVZ Zeiten:",
          sorted([t for t in sample_times if 360 <= t <= 540])[:20])


# -------------------------------------------------
# 11. Debug: Wo fehlen noch Headways?
# -------------------------------------------------
no_headway_mask = gdf["headway_avg"].isna()

print("Adressen ohne Headway_avg:", int(no_headway_mask.sum()))

# Welche Stop-IDs sind schuld, nach nearest_stop_id
na_stops = (
    gdf.loc[no_headway_mask, "nearest_stop_id"]
    .value_counts()
)
print("Anzahl unterschiedlicher problematischer Haltestellen (Steig-Ebene):", len(na_stops))
print(na_stops.head(20))

# Welche Cluster sind schuld
na_clusters = (
    gdf.loc[no_headway_mask, "pt_cluster_id"]
    .value_counts()
)
print("Anzahl problematischer Cluster:", len(na_clusters))
print(na_clusters.head(20))

# Prüfen, ob diese Cluster überhaupt Headways in df_headways haben
clusters_with_headway = set(df_headways["pt_cluster_id"].astype(int).tolist())
clusters_without = [cid for cid in na_clusters.index if cid not in clusters_with_headway]
print("Cluster ohne Eintrag in df_headways:", len(clusters_without))
print(clusters_without[:20])

# Sanity check: für einen der Top-Cluster, zeig alle Abfahrtszeiten morgens
if len(clusters_without) > 0:
    test_cluster = clusters_without[0]
    sample_times = stopdata.loc[stopdata["pt_cluster_id"] == test_cluster, "minutes"]
    print("Beispiel-Cluster", test_cluster, "hat morgens Zeiten in 6-9?:",
          sample_times[(sample_times>=360)&(sample_times<=540)].sort_values().head(20).tolist())

# -------------------------------------------------
# 12. Cleanup (optional)
# -------------------------------------------------
#del(stop_times, trips, calendar, trips_filtered,
#    stopdata, df_hm, df_he, df_headways,
#    gdf_stops_metric, coords, db)


In [ ]:
# ÖPMV in gdf mergen
headway_attribute = ["headway_avg"]

## Visualisierung der ÖPNV-Taktung

In [ ]:
import folium
from branca.colormap import linear, LinearColormap
import pandas as pd
import numpy as np

# Convert gdf_stops geometry to WGS84
gdf_stops = gdf_stops.to_crs(epsg=4326).copy()

# Haltestellen innerhalb Stadt
gdf_stops_clip = gdf_stops[gdf_stops.geometry.within(CITY_BOUNDING_BOX)].copy()
print("Haltestellen im Stadtpolygon:", len(gdf_stops_clip))

#
# 2. Farbskala nur aus Adressen-Headway
#
# headway_avg = durchschnittliche Taktzeit (Minuten) für diese Adresse
# Annahme: kleiner Wert = besser (häufigere Bedienung)
addr_headway = pd.to_numeric(gdf["headway_avg"], errors="coerce")

hv_valid = addr_headway.dropna()
if len(hv_valid) > 0:
    vmin, vmax = hv_valid.quantile([0.01, 0.99])
    if vmin == vmax:
        # falls alles gleich (z. B. nur eine Linie), spreizen für die Farbskala
        vmin = vmin - 0.1
        vmax = vmax + 0.1
else:
    # Fallback, falls ALLE Adressen NaN sind
    vmin, vmax = (0, 1)

palette_normal = list(linear.RdYlGn_11.colors)
palette_inverted = palette_normal[::-1]
colormap = LinearColormap(
    colors=palette_inverted,
    vmin=vmin,
    vmax=vmax,
).to_step(n=9)
colormap.caption = "Headway pro Adresse (Minuten, kleiner = besser)"

# 3. Karte initialisieren
m = folium.Map(location=CITY_CENTER, zoom_start=13, tiles="cartodbpositron")

# 4. Adressen plotten (farbig nach headway_avg)
#    - Farbig wenn headway_avg da
#    - Hellgrau wenn kein Wert
for _, row in gdf.iterrows():
    hv_addr = row.get("headway_avg", np.nan)
    hv_morning = row.get("headway_morning", np.nan)
    hv_evening = row.get("headway_evening", np.nan)

    if pd.isna(hv_addr):
        # Kein Wert berechnet -> zeichne neutral
        color = "#BBBBBB"
        fill_color = "#BBBBBB"
        hv_label = "kein Wert"
    else:
        color = colormap(hv_addr)
        fill_color = colormap(hv_addr)
        hv_label = f"{hv_addr:.1f} min"

    # Popup mit allen Headways, falls vorhanden
    popup_lines = [
        f"{row.get('Straßenname', '')} {row.get('Hsnr', '')}",
        f"<b>Headway (avg):</b> {hv_label}",
    ]
    if pd.notna(hv_morning):
        popup_lines.append(f"Frühspitze: {hv_morning:.1f} min")
    if pd.notna(hv_evening):
        popup_lines.append(f"Abendspitze: {hv_evening:.1f} min")

    popup_html = "<br>".join(popup_lines)

    folium.CircleMarker(
        location=[row.lat, row.lon],
        radius=5,
        color=color,
        fill=True,
        fill_color=fill_color,
        fill_opacity=0.8,
        weight=1,
        popup=popup_html,
    ).add_to(m)

# 5. Haltestellen plotten (schwarz, neutral)
#    Du zeigst hier die Infrastrukturpunkte, ohne Qualitätsfarbe.
for _, row in gdf_stops_clip.iterrows():
    lat_s = row["stop_lat"]
    lon_s = row["stop_lon"]

    # Popup Haltestellenname
    stop_label = row.get("stop_name", row.get("stop_id", "Haltestelle"))

    folium.CircleMarker(
        location=[lat_s, lon_s],
        radius=3,
        color="black",
        fill=True,
        fill_color="black",
        fill_opacity=1,
        weight=1,
        popup=f"<b>Haltestelle:</b> {stop_label}<br>",
    ).add_to(m)

# 6. Legende für die Adressen-Headways
colormap.add_to(m)
m


## Umwelt
### Großflächen
- Luftlinie in m zum nächsten großen Wald oder See
- Luftlinie in m zum nächsten großen Gewerbe- oder Industriegebiet


In [ ]:
TARGET_CLASSES = ["Gewerbe", "Gruen", "Sonstiges"]

# --- 1) metrische Projektion ---
gdf_m = gdf.to_crs(32633)
gdf_large_m = gdf_large.to_crs(32633)

# --- 2) Zentroid ---
gdf_large_m["centroid"] = gdf_large_m.geometry.centroid

# --- 3) Robuste Distanzfunktion ---
import numpy as np

def compute_min_distance(point, target_series):
    sindex = target_series.sindex
    idx = sindex.nearest(point)

    # ndarray → normaler Fall unter RTree
    if isinstance(idx, (list, tuple, np.ndarray)):
        if len(idx) == 0:
            return None
        nearest_idx = idx[0]

    elif isinstance(idx, tuple):
        nearest_idx = idx[0][0]

    else:
        raise TypeError(f"Unexpected type from sindex.nearest: {type(idx)}")

    nearest_geom = target_series.iloc[nearest_idx]
    return point.distance(nearest_geom)

# --- 4) große Flächen pro Klasse sammeln ---
large_by_class = {
    k: gdf_large_m[gdf_large_m["nutzklasse"] == k]["centroid"]
    for k in TARGET_CLASSES
}

# --- 5) Distanz berechnen ---
for k, centroids in large_by_class.items():
    gdf_m[f"dist_gross_{k.lower()}"] = gdf_m.geometry.apply(
        lambda pt, centroids=centroids: compute_min_distance(pt, centroids)
    )

# --- 6) zurück in WGS84 ---
gdf_back = gdf_m.to_crs(4326)

# --- 7) neue Distanzfelder zurück in das ursprüngliche gdf mergen ---
dist_cols = [f"dist_gross_{k.lower()}" for k in TARGET_CLASSES]
gdf[dist_cols] = gdf_back[dist_cols]

del gdf_back

### Freizeit- und Erholungsflächen

- Fußläufige Distanz zur nächstgelegenen Freizeit- und Erholungsflächen (Parks, Grünanlagen)

Die Freizeit- und Erholungsflächen laut Auftrag sind:
- Marienberg
- Humboldthain
- Salzhofufer
- Wallpromenade
- Theaterpark
- Grabenpromenade
- Schlosspark Plaue
- Schlosspark Gollwitz
- Krugpark

Aus den bereitgestellten Shape-Dateien werden anhand der Objektbezeichnung (Spalte "objektbeze") im Routing-Skript ca. 100 zusammenhängende Flächen gebildet, deren Ränder als Ziele für die fußläufige Distanzberechnung genutzt werden. Dadurch werden mehr als die angegebenen Flächen genutzt.

Einschränkung der Validität: Durch die Beschränkung auf diese Flächenarten erhalten zentrumsferne Adressen trotz der Lage z. B. an Wäldern schlechtere Bewertungen.

In [ ]:
path = "data/Grünflächen_Verkehrszeichen/20251029_Vegetation_KSP_GP_31.shp"
gdf_gruen_shape = gpd.read_file(path)
#gdf_gruen

In [ ]:
import folium

m = folium.Map(location=CITY_CENTER, zoom_start=14)

def random_color(seed):
    rng = np.random.default_rng(seed)
    r = rng.integers(80, 200)
    g = rng.integers(80, 200)
    b = rng.integers(80, 200)
    return f"#{r:02x}{g:02x}{b:02x}"

# Ein Farbschema erzeugen
unique_ids = gdf_gruen_shape["objektbeze"].unique()
color_map = {uid: random_color(i) for i, uid in enumerate(unique_ids)}

# Polygone hinzufügen
folium.GeoJson(
    gdf_gruen_shape,
    style_function=lambda feature: {
        "color": "black",
        "weight": 0.5,
        "fillOpacity": 0.9,
        "fillColor": color_map[feature["properties"]["objektbeze"]]
    },
    tooltip=folium.GeoJsonTooltip(fields=["objektbeze"])
).add_to(m)

m

In [ ]:
gdf_gruen_m = gdf_gruen_shape.to_crs(32633)

# Gruppieren & union der Flächen
gdf_gruen_area = (gdf_gruen_shape.dissolve(by="objektbeze").reset_index())

# Fläche berechnen (m²)
gdf_gruen_area["flaeche_m2"] = gdf_gruen_area.area
gdf_gruen_area["flaeche_ha"] = gdf_gruen_area["flaeche_m2"] / 10_000

bad = gdf_gruen_area.geometry.apply(lambda g: not g.is_valid)
print("Ungültige Geometrien:", bad.sum())

bb = gdf_gruen_area.geometry.boundary
print("Boundary is empty:", sum(bb.is_empty))

# Optional: Die Fläche der nächstgelegenen Anlage könnte noch als Qualitätskriterium hinzugefügt werden. Vorerst nutzen wir nur die Entfernung zu den Anlagen.

del(gdf_gruen_m, gdf_gruen_area)

In [ ]:
# Lade vorberechnete Routen zur nächstgelegenen Anlage und Anzahl von Funden im Umkreis
gdf_gruen = load_geocsv("out/adressen_mit_gruen_routen.csv")
gdf_gruen

In [ ]:
# Merge mit Haupt-GDF
gdf_gruen["Adresse_merge"] = gdf_gruen.apply(make_merge_addr, axis=1)
gdf_gruen = gdf_gruen.sort_values("Adresse_merge").drop_duplicates("Adresse_merge", keep="first")

dups = gdf_gruen["Adresse_merge"].value_counts()
print(dups[dups > 1])

gruen_attribute = ["gruen_route","gruen_min_distance_m", "gruen_count_within_500m", "gruen_count_within_800m", "gruen_count_within_1000m"]

gdf = pd.merge(
    gdf,
    gdf_gruen[["Adresse_merge"] + gruen_attribute],
    on="Adresse_merge",
    how="left",
    validate="one_to_many"
)

# Plausibiliätsprüfung
print(gdf[["Adresse_merge", "gruen_min_distance_m", "gruen_route"]].head())
print(gdf.shape)

## Begenzung durch dauerhafte Bahnlininen / Schranken
Als zusätzlicher Indikator wird berechnet, ob eine Adresse durch eine Bahnlinie oder eine stark befahrene Straße (Autobahn, Bundesstraße) vom Stadtzentrum getrennt ist. Damit wird abgebildet, ob eine Adresse trotz Nähe zum Zentrum durch eine Barriere erschwerten Zugang hat (z. B., durch Wartezeiten an Bahnübergängen).

In [ ]:
import osmnx as ox

# Schritt 1: Bahnlinien von OSM laden

place = "Brandenburg an der Havel, Germany"

# Eisenbahnlinien holen
rails = ox.features_from_place(
    place,
    tags={"railway": True}  # includes rail, light_rail, tram, etc.
)

# nur Schienenverkehr (keine Haltestellen)
rails = rails[rails["railway"].isin(["rail"])]

In [ ]:

from shapely.geometry import LineString

# Schritt 2: Fußĺäufige Routen zum Stadtzentrum auf Schnittpunkt mit Bahnlinie prüfen und im Datensatz speichern

def route_crosses_rail(route_geojson_str, rail_geom):
    if not isinstance(route_geojson_str, str):
        return False

    try:
        geo = json.loads(route_geojson_str)
    except:
        return False

    if geo.get("type") != "LineString":
        return False

    # Koordinaten (lon, lat) → Shapely LineString nutzen
    coords = geo["coordinates"]
    line = LineString(coords)

    # Schnitt-Test
    return line.intersects(rail_geom)

rail_union = rails.union_all()
gdf["behind_rail_from_center"] = gdf["center_route"].apply(
    lambda r: int(route_crosses_rail(r, rail_union))
)


In [ ]:

import json
import folium

# 10 zufällige Adressen ziehen
sample_gdf = gdf.sample(10, random_state=42)

center_lat, center_lon = CITY_CENTER  # CITY_CENTER ist [lat, lon]
center_point = (center_lat, center_lon)
m = folium.Map(location=CITY_CENTER, zoom_start=13, tiles="cartodbpositron")

# Zentrum markieren
folium.Marker(
    location=CITY_CENTER,
    tooltip="Stadtzentrum",
    icon=folium.Icon(color="red", icon="star")
).add_to(m)

# Route + Marker je Adresse
for idx, row in sample_gdf.iterrows():
    if pd.isna(row["lat"]) or pd.isna(row["lon"]):
        continue

    addr_point = (row["lat"], row["lon"])
    behind_flag = row.get("behind_rail_from_center", None)

    tooltip = f"Adresse {idx}<br>behind_rail_from_center: {behind_flag}"

    # Route zum Zentrum (GeoJSON)
    route_json = row.get("center_route")

    if isinstance(route_json, str):
        try:
            route_geo = json.loads(route_json)

            if route_geo.get("type") == "LineString":
                # GeoJSON Koordinaten sind (lon, lat)
                coords = [(c[1], c[0]) for c in route_geo["coordinates"]]

                folium.PolyLine(
                    locations=coords,
                    color="blue",
                    weight=4,
                    opacity=0.7,
                    tooltip=f"Route zu Adresse {idx}"
                ).add_to(m)

        except Exception as e:
            print(f"Fehler beim Zeichnen der Route für Adresse {idx}: {e}")

    # Adressmarker
    folium.CircleMarker(
        location=addr_point,
        radius=4,
        color="blue",
        fill=True,
        fill_opacity=0.8,
        tooltip=folium.Tooltip(tooltip)
    ).add_to(m)

# Bahnlinien-Layer
bahn_layer = folium.FeatureGroup(name="Bahnlinien (OSM)", show=True)

for _, row in rails.iterrows():
    geom = row.geometry
    if geom is None:
        continue

    if geom.geom_type == "LineString":
        coords = [(y, x) for x, y in geom.coords]
        folium.PolyLine(
            locations=coords,
            color="darkred",
            weight=4,
            opacity=0.8
        ).add_to(bahn_layer)

    elif geom.geom_type == "MultiLineString":
        for part in geom:
            coords = [(y, x) for x, y in part.coords]
            folium.PolyLine(
                locations=coords,
                color="darkred",
                weight=4,
                opacity=0.8
            ).add_to(bahn_layer)

bahn_layer.add_to(m)

m

# Visualisierung aller Indikatoren
Alle Einflussfaktoren (Supermärkte, Ärzte, Schulen etc.) werden zur Plausibilitätsprüfung auf einer Karte visualisiert.

In [ ]:
import folium
import branca.colormap as cm
from helper import add_markers_from_csv, STRASSENNAME, HAUSNUMMER, HAUSNUMMERZUSATZ

m = folium.Map(location=CITY_CENTER, zoom_start=13, tiles="cartodbpositron")

# Add layer markers
add_markers_from_csv(map_obj=m, csv_path="out/einzelhandel_geocoded.csv", color="blue", icon="shopping-cart", layer_name="Einzelhandel")
add_markers_from_csv(map_obj=m, csv_path="out/grundschulen_geocoded.csv", color="green", icon="graduation-cap", layer_name="Grundschulen")
add_markers_from_csv(map_obj=m, csv_path="out/kitas_geocoded.csv", color="beige", icon="child", layer_name="Kitas")
add_markers_from_csv(map_obj=m, csv_path="out/haltestellen_geocoded.csv", color="lightgray", icon="bus", layer_name="Haltestellen")
add_markers_from_csv(map_obj=m, csv_path="out/medzentren_geocoded.csv", color="red", icon="staff-snake", layer_name="Medizinische Zentren")

# Lärmindex aus dem Geopackage
gdf_laerm_karte = gpd.read_file("data/laerm.gpkg", layer="laerm")
gdf_laerm_karte = gdf_laerm_karte.clip(CITY_BOUNDING_BOX)
value_column = "isov1"
min_val = gdf_laerm_karte[value_column].min()
max_val = gdf_laerm_karte[value_column].max()
colormap = cm.LinearColormap(colors=["green", "yellow", "red"], vmin=min_val, vmax=max_val)
colormap.caption = "Lärmpegel (LDEN in dB)"

# Alle Adressen als Punkte
adress_layer = folium.FeatureGroup(name="Wohnadressen", show=False)
for _, row in gdf.iterrows():
    if pd.notna(row["lat"]) and pd.notna(row["lon"]):
        strasse = s(row.get(STRASSENNAME))
        hsnr = s(row.get(HAUSNUMMER))
        hsnrzus = s(row.get(HAUSNUMMERZUSATZ))

        tooltip = strasse + " " + hsnr

        adressen_map = folium.CircleMarker(
            location=[row["lat"], row["lon"]],
            radius=3,
            color="lightgray",
            fill=True,
            fill_opacity=0.6,
            tooltip=tooltip
        ).add_to(adress_layer)
adress_layer.add_to(m)

def style_function(feature):
    value = feature["properties"][value_column]
    return {
        "fillColor": colormap(value),
        "color": "black",
        "weight": 0.3,
        "fillOpacity": 0.2
    }

laerm_layer = folium.FeatureGroup(name="Lärmkarte (LDEN 2022)")
folium.GeoJson(
    gdf_laerm_karte,
    style_function=style_function,
).add_to(laerm_layer)
laerm_layer.add_to(m)
colormap.add_to(m)

# Grünflächen hinzufügen
gruen_layer = folium.FeatureGroup(name="Freizeit- und Erholungsflächen")
folium.GeoJson(
    gdf_gruen_shape,
    style_function=lambda feature: {
        "color": "black",
        "weight": 0.2,
        "fillOpacity": 0.9,
        "fillColor": "green"
    }
).add_to(gruen_layer)
gruen_layer.add_to(m)

# Bahnlinien layer hinzufügen
bahn_layer = folium.FeatureGroup(name="Bahnlinien (OSM)", show=True)

for _, row in rails.iterrows():
    geom = row.geometry
    if geom is None:
        continue

    if geom.geom_type == "LineString":
        coords = [(y, x) for x, y in geom.coords]
        folium.PolyLine(
            locations=coords,
            color="darkred",
            weight=4,
            opacity=0.8
        ).add_to(bahn_layer)

    elif geom.geom_type == "MultiLineString":
        for part in geom:
            coords = [(y, x) for x, y in part.coords]
            folium.PolyLine(
                locations=coords,
                color="darkred",
                weight=4,
                opacity=0.8
            ).add_to(bahn_layer)

bahn_layer.add_to(m)

# Schaltbare Layer
folium.LayerControl(collapsed=False).add_to(m)

del gdf_laerm_karte  # Speicher freigeben

m


# Scoring / Punktesystem
Hier werden z-Werte zu alle **Einflussfaktoren** gebildet, um die Abweichung einer Ausprägung vom Standard (im betrachteten Gebiet) zu erfassen. Weiterhin werden die **Gewichte** festgelegt, mit denen die Faktoren in die Wohnlagenbewertung eingehen.

In [ ]:
from scipy.stats import zscore

# ---------------------------------------
# 1) Numerische & binäre Features
# ---------------------------------------
numeric_features = (
        ["center_distance_m"] +
        haltestellen_attribute +
        headway_attribute +
        einzelhandel_attribute +
        laerm_attribute +
        kitas_attribute +
        grundschulen_attribute +
        medzentren_attribute +
        gruen_attribute
)

binary_features = ["behind_rail_from_center"]

score_vars = numeric_features + binary_features

# Masken nur auf numerische Daten!
mask_all = gdf[numeric_features].notna().all(axis=1)

# ---------------------------------------
# 2) Z-Scores für numerische Variablen
#    (immer: hoch = gut!)
# ---------------------------------------

# Zentralität
gdf.loc[mask_all, "z_centrality"] = -zscore(gdf.loc[mask_all, "center_distance_m"])

# Einzelhandel
gdf.loc[mask_all, "z_einzelhandel_distance"]    = -zscore(gdf.loc[mask_all, "einzelhandel_min_distance_m"])
gdf.loc[mask_all, "z_einzelhandel_near_500"]    =  zscore(gdf.loc[mask_all, "einzelhandel_500m_count"])
gdf.loc[mask_all, "z_einzelhandel_near_800"]    =  zscore(gdf.loc[mask_all, "einzelhandel_800m_count"])
gdf.loc[mask_all, "z_einzelhandel_near_1000"]   =  zscore(gdf.loc[mask_all, "einzelhandel_1000m_count"])

# Lärm
gdf.loc[mask_all, "z_laerm_index_tag"]          = -zscore(gdf.loc[mask_all, "laerm_index_tag"])

# Kitas
gdf.loc[mask_all, "z_kita_distance"]            = -zscore(gdf.loc[mask_all, "kitas_min_distance_m"])
gdf.loc[mask_all, "z_kita_near_500"]            =  zscore(gdf.loc[mask_all, "kitas_count_within_500m"])
gdf.loc[mask_all, "z_kita_near_800"]            =  zscore(gdf.loc[mask_all, "kitas_count_within_800m"])
gdf.loc[mask_all, "z_kita_near_1000"]           =  zscore(gdf.loc[mask_all, "kitas_count_within_1000m"])

# Grundschulen
gdf.loc[mask_all, "z_grundschulen_distance"]    = -zscore(gdf.loc[mask_all, "grundschulen_min_distance_m"])
gdf.loc[mask_all, "z_grundschulen_near_500"]    =  zscore(gdf.loc[mask_all, "grundschulen_count_within_500m"])
gdf.loc[mask_all, "z_grundschulen_near_800"]    =  zscore(gdf.loc[mask_all, "grundschulen_count_within_800m"])
gdf.loc[mask_all, "z_grundschulen_near_1000"]   =  zscore(gdf.loc[mask_all, "grundschulen_count_within_1000m"])

# Mobilität
gdf.loc[mask_all, "z_haltestelle_distance"]     = -zscore(gdf.loc[mask_all, "haltestellen_min_distance_m"])
gdf.loc[mask_all, "z_haltestellen_count_within_500m"] =  zscore(gdf.loc[mask_all, "haltestellen_count_within_500m"])
gdf.loc[mask_all, "z_haltestellen_count_within_800m"] =  zscore(gdf.loc[mask_all, "haltestellen_count_within_800m"])
gdf.loc[mask_all, "z_headway_score"]            = -zscore(gdf.loc[mask_all, "headway_avg"])

# Medizinische Versorgung
gdf.loc[mask_all, "z_medzentrum_distance"]      = -zscore(gdf.loc[mask_all, "medzentren_min_distance_m"])
gdf.loc[mask_all, "z_medzentrum_near_500"]      =  zscore(gdf.loc[mask_all, "medzentren_count_within_500m"])
gdf.loc[mask_all, "z_medzentrum_near_800"]      =  zscore(gdf.loc[mask_all, "medzentren_count_within_800m"])
gdf.loc[mask_all, "z_medzentrum_near_1000"]     =  zscore(gdf.loc[mask_all, "medzentren_count_within_1000m"])

# Grünflächen
gdf.loc[mask_all, "z_gruen_distance"]           = -zscore(gdf.loc[mask_all, "gruen_min_distance_m"])
gdf.loc[mask_all, "z_gruen_near_500"]           =  zscore(gdf.loc[mask_all, "gruen_count_within_500m"])
gdf.loc[mask_all, "z_gruen_near_800"]           =  zscore(gdf.loc[mask_all, "gruen_count_within_800m"])
gdf.loc[mask_all, "z_gruen_near_1000"]          =  zscore(gdf.loc[mask_all, "gruen_count_within_1000m"])

# Gewerbe (Industrie) -> weiter weg = gut
mask_mm_gewerbe = gdf["dist_gross_gewerbe"].notna()
gdf.loc[mask_mm_gewerbe, "z_dist_gross_gewerbe"] = zscore(gdf.loc[mask_mm_gewerbe, "dist_gross_gewerbe"])

# Gruen -> näher = gut
mask_mm_gruen = gdf["dist_gross_gruen"].notna()
gdf.loc[mask_mm_gruen, "z_dist_gross_gruen"]    = -zscore(gdf.loc[mask_mm_gruen, "dist_gross_gruen"])

# Sonstiges (Wasser) -> näher = gut
mask_mm_sonst = gdf["dist_gross_sonstiges"].notna()
gdf.loc[mask_mm_sonst, "z_dist_gross_sonstiges"] = -zscore(gdf.loc[mask_mm_sonst, "dist_gross_sonstiges"])


# ---------------------------------------
# 3) Score-Dimensionen (alle Summen = 1.0)
# ---------------------------------------

### 3.1 Zentralität (1 Dimension, Summe = 1.0)
mask_mm_central = gdf["center_distance_m"].notna()
gdf.loc[mask_mm_central, "score_zentralitaet"] = gdf.loc[mask_mm_central, "z_centrality"]


### 3.2 Versorgung (Summe = 1.0)
mask_mm_versorgung = gdf[[
    "einzelhandel_min_distance_m",
    "einzelhandel_500m_count",
    "einzelhandel_800m_count",
    "einzelhandel_1000m_count",
    "medzentren_min_distance_m",
    "medzentren_count_within_500m",
    "medzentren_count_within_800m",
    "medzentren_count_within_1000m"
]].notna().all(axis=1)

gdf.loc[mask_mm_versorgung, "score_versorgung"] = (
      0.15 * gdf.loc[mask_mm_versorgung, "z_einzelhandel_distance"]
    + 0.10 * gdf.loc[mask_mm_versorgung, "z_einzelhandel_near_500"]
    + 0.10 * gdf.loc[mask_mm_versorgung, "z_einzelhandel_near_800"]
    + 0.10 * gdf.loc[mask_mm_versorgung, "z_einzelhandel_near_1000"]
    + 0.15 * gdf.loc[mask_mm_versorgung, "z_medzentrum_distance"]
    + 0.10 * gdf.loc[mask_mm_versorgung, "z_medzentrum_near_500"]
    + 0.10 * gdf.loc[mask_mm_versorgung, "z_medzentrum_near_800"]
    + 0.10 * gdf.loc[mask_mm_versorgung, "z_medzentrum_near_1000"]
)


### 3.3 Mobilität (Summe = 1.0)
mask_mm_mobilitaet = gdf[[
    "headway_avg",
    "haltestellen_min_distance_m",
    "haltestellen_count_within_500m",
    "haltestellen_count_within_800m"
]].notna().all(axis=1)

gdf.loc[mask_mm_mobilitaet, "score_mobilitaet"] = (
      0.40 * gdf.loc[mask_mm_mobilitaet, "z_headway_score"]
    + 0.40 * gdf.loc[mask_mm_mobilitaet, "z_haltestelle_distance"]
    + 0.10 * gdf.loc[mask_mm_mobilitaet, "z_haltestellen_count_within_500m"]
    + 0.10 * gdf.loc[mask_mm_mobilitaet, "z_haltestellen_count_within_800m"]
)


### 3.4 Bildung (Summe = 1.0)
mask_mm_bildung = gdf[[
    "kitas_min_distance_m",
    "kitas_count_within_500m",
    "kitas_count_within_800m",
    "kitas_count_within_1000m",
    "grundschulen_min_distance_m",
    "grundschulen_count_within_500m",
    "grundschulen_count_within_800m",
    "grundschulen_count_within_1000m"
]].notna().all(axis=1)

gdf.loc[mask_mm_bildung, "score_bildung"] = (
      0.15 * gdf.loc[mask_mm_bildung, "z_kita_distance"]
    + 0.10 * gdf.loc[mask_mm_bildung, "z_kita_near_500"]
    + 0.10 * gdf.loc[mask_mm_bildung, "z_kita_near_800"]
    + 0.10 * gdf.loc[mask_mm_bildung, "z_kita_near_1000"]
    + 0.15 * gdf.loc[mask_mm_bildung, "z_grundschulen_distance"]
    + 0.10 * gdf.loc[mask_mm_bildung, "z_grundschulen_near_500"]
    + 0.10 * gdf.loc[mask_mm_bildung, "z_grundschulen_near_800"]
    + 0.10 * gdf.loc[mask_mm_bildung, "z_grundschulen_near_1000"]
)


### 3.5 Umwelt (Summe = 1.0)
# Umwelt (Summe = 1.0)
mask_mm_umwelt = gdf[[
    "gruen_min_distance_m",
    "gruen_count_within_500m",
    "gruen_count_within_800m",
    "gruen_count_within_1000m",
    "laerm_index_tag",
    "z_dist_gross_gewerbe",
    "z_dist_gross_gruen",
    "z_dist_gross_sonstiges"
]].notna().all(axis=1)

gdf.loc[mask_mm_umwelt, "score_umwelt"] = (
      0.30 * gdf.loc[mask_mm_umwelt, "z_gruen_distance"]
    + 0.10 * gdf.loc[mask_mm_umwelt, "z_gruen_near_500"]
    + 0.10 * gdf.loc[mask_mm_umwelt, "z_gruen_near_800"]
    + 0.05 * gdf.loc[mask_mm_umwelt, "z_gruen_near_1000"]
    + 0.15 * gdf.loc[mask_mm_umwelt, "z_laerm_index_tag"]
    + 0.15 * gdf.loc[mask_mm_umwelt, "z_dist_gross_gewerbe"]     # Industrie
    + 0.10 * gdf.loc[mask_mm_umwelt, "z_dist_gross_gruen"]       # Grün
    + 0.05 * gdf.loc[mask_mm_umwelt, "z_dist_gross_sonstiges"]   # Wasser
)

# ---------------------------------------
# 4) Gesamt-Score (alle Dimensionen verfügbar)
# ---------------------------------------
score_all_vars = [
    "score_zentralitaet",
    "score_bildung",
    "score_versorgung",
    "score_umwelt",
    "score_mobilitaet"
]

mask_all_scores = gdf[score_all_vars].notna().all(axis=1)

gdf.loc[mask_all_scores, "score_total"] = (
      0.20 * gdf.loc[mask_all_scores, "score_zentralitaet"]
    + 0.20 * gdf.loc[mask_all_scores, "score_bildung"]
    + 0.20 * gdf.loc[mask_all_scores, "score_versorgung"]
    + 0.20 * gdf.loc[mask_all_scores, "score_umwelt"]
    + 0.20 * gdf.loc[mask_all_scores, "score_mobilitaet"]
)

print("Anzahl gültiger Gesamt-Scores:", gdf["score_total"].notna().sum())


# Validierung

Im Folgenden werden die Z-Variablen genutzt, um mittels K-Means-Clustering Wohnlagen zu identifizieren. Zunächst wird die optimale Clusteranzahl mittels Elbow-Methode und Silhouetten-Analyse bestimmt. Danach wird das finale K-Means-Modell mit der gewählten Clusteranzahl trainiert und die Wohnlagen den Adressen zugewiesen.

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# ---------------------------------------
# Z-Variablen aus allen Kategorien
# ---------------------------------------
z_vars = [
    # Zentralität
    "z_centrality",

    # Einzelhandel
    "z_einzelhandel_distance",
    "z_einzelhandel_near_500",
    "z_einzelhandel_near_800",
    "z_einzelhandel_near_1000",

    # Lärm
    "z_laerm_index_tag",

    # Kitas
    "z_kita_distance",
    "z_kita_near_500",
    "z_kita_near_800",
    "z_kita_near_1000",

    # Grundschulen
    "z_grundschulen_distance",
    "z_grundschulen_near_500",
    "z_grundschulen_near_800",
    "z_grundschulen_near_1000",

    # Haltestellen / Headway
    "z_haltestelle_distance",
    "z_haltestellen_count_within_500m",
    "z_haltestellen_count_within_800m",
    "z_headway_score",

    # MedZentren
    "z_medzentrum_distance",
    "z_medzentrum_near_500",
    "z_medzentrum_near_800",
    "z_medzentrum_near_1000",

    # Grünflächen
    "z_gruen_distance",
    "z_gruen_near_500",
    "z_gruen_near_800",
    "z_gruen_near_1000",

    # Distanzen zu Großflächen
    "z_dist_gross_gewerbe",
    "z_dist_gross_gruen",
    "z_dist_gross_sonstiges",

    # Barriere
    "behind_rail_from_center"
]

# ---------------------------------------
# Validierung
# ---------------------------------------
missing = [c for c in z_vars if c not in gdf.columns]
if missing:
    raise ValueError(f"Diese Z-Variablen fehlen im gdf: {missing}")

# ---------------------------------------
# Datenmatrix: nur vollständige Zeilen
# ---------------------------------------
X = gdf[z_vars].dropna().values

# ---------------------------------------
# Elbow-Methode
# ---------------------------------------
inertia = []
cluster_range = range(2, 20)

for k in cluster_range:
    model = KMeans(n_clusters=k, random_state=42)
    model.fit(X)
    inertia.append(model.inertia_)

plt.figure(figsize=(8,5))
plt.plot(cluster_range, inertia, marker='o')
plt.title("Elbow-Methode: KMeans-Inertia vs. Clusteranzahl")
plt.xlabel("Anzahl Cluster (k)")
plt.ylabel("Inertia (Distanz innerhalb der Cluster)")
plt.xticks(cluster_range)
plt.grid(True)
plt.show()

In [ ]:
# Silhouetten-Analyse
from sklearn.metrics import silhouette_score

silhouettes = []

for k in cluster_range:
    model = KMeans(n_clusters=k, random_state=42)
    labels = model.fit_predict(X)
    score = silhouette_score(X, labels, random_state=42)
    silhouettes.append(score)

plt.figure(figsize=(8,5))
plt.plot(cluster_range, silhouettes, marker='o', color="green")
plt.title("Silhouetten-Score pro Clusteranzahl")
plt.xlabel("Anzahl Cluster (k)")
plt.ylabel("Durchschn. Silhouetten-Koeffizient")
plt.xticks(cluster_range)
plt.grid(True)
plt.show()


In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

# ----------------------------
# 1) KMeans
# ----------------------------
X = gdf[z_vars].dropna().values  # Nur vollständige Zeilen

# Die Anzahl der Cluster muss anhand der obigen Scores eingeschätzt und festgelegt werden
NUMBER_OF_CLUSTERS = 13

model = KMeans(n_clusters=NUMBER_OF_CLUSTERS, random_state=42).fit(X)

mask_complete = gdf[z_vars].notna().all(axis=1)
gdf.loc[mask_complete, "cluster"] = model.labels_

cluster_centers = pd.DataFrame(model.cluster_centers_, columns=z_vars)
cluster_centers.index.name = "Cluster"
print(cluster_centers.round(2))


# ----------------------------
# 2) PCA (3D + 2D)
# ----------------------------
X_scaled = X  # Z-Scores → keine Skalierung nötig

# 3 Komponenten für 3D-Plot
pca3 = PCA(n_components=3, random_state=42)
X_pca3 = pca3.fit_transform(X_scaled)
print("Explained variance ratio (3 PCs):", pca3.explained_variance_ratio_)

# ----------------------------
# 3) 3D-PCA Scatterplot
# ----------------------------
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

cmap = plt.get_cmap("tab10")

scatter = ax.scatter(
    X_pca3[:, 0],
    X_pca3[:, 1],
    X_pca3[:, 2],
    c=model.labels_,
    cmap=cmap,
    s=20,
    alpha=0.6
)

# Optional: Cluster-Zentren im PCA Raum
centers_pca3 = pca3.transform(cluster_centers.to_numpy())
ax.scatter(
    centers_pca3[:, 0],
    centers_pca3[:, 1],
    centers_pca3[:, 2],
    c=range(NUMBER_OF_CLUSTERS),
    cmap=cmap,
    s=200,
    marker="X",
    edgecolor="black"
)

ax.set_title("KMeans-Cluster (3D PCA Projektion)")
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")
plt.show()


# ----------------------------
# 4) Klassischer 2D PCA Plot
# ----------------------------
pca2 = PCA(n_components=2, random_state=42)
X_pca2 = pca2.fit_transform(X_scaled)
print("Explained variance ratio (2 PCs):", pca2.explained_variance_ratio_)

plt.figure(figsize=(8,6))
plt.scatter(X_pca2[:,0], X_pca2[:,1], c=model.labels_, cmap=cmap, alpha=0.6)
plt.title("KMeans-Cluster (PCA 2D-Projektion)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()


In [ ]:
import folium

gdf = gdf[gdf["lat"].notna() & gdf["lon"].notna() & gdf["cluster"].notna()]
gdf["cluster"] = gdf["cluster"].astype(int)

# Farbpalette für Cluster
cluster_colors = {
    0:  "#e41a1c",  # kräftiges Rot
    1:  "#377eb8",  # kräftiges Blau
    2:  "#4daf4a",  # kräftiges Grün
    3:  "#984ea3",  # kräftiges Lila
    4:  "#ff7f00",  # Orange
    5:  "#ffff33",  # Gelb (klar sichtbar)
    6:  "#a65628",  # Braun
    7:  "#f781bf",  # Pink
    8:  "#999999",  # Mittelgrau
    9:  "#1b9e77",  # türkis-grün (klar unterscheidbar von grün)
    10: "#d95f02",  # warmes Orange (andere Familie als #ff7f00)
    11: "#7570b3",  # Blau-Lila (gut unterscheidbar von 1 & 3)
    12: "#e7298a",  # Magenta
    13: "#66a61e",  # Olivgrün (Matt → weit von #4daf4a)
    14: "#e6ab02",  # Senfgelb
    15: "#a6761d",  # Ocker
    16: "#666666",  # Dunkelgrau
    17: "#1f78b4",  # Dunkelblau (kräftig unterscheidbar)
    18: "#b2df8a",  # helles, aber eindeutig blasses Grün
    19: "#6a3d9a",  # Dunkellila
}

m = folium.Map(location=CITY_CENTER, zoom_start=13, tiles="cartodbpositron")

# Add layer markers
add_markers_from_csv(map_obj=m, csv_path="out/einzelhandel_geocoded.csv", color="blue", icon="shopping-cart", layer_name="Einzelhandel")
add_markers_from_csv(map_obj=m, csv_path="out/grundschulen_geocoded.csv", color="green", icon="graduation-cap", layer_name="Grundschulen")
add_markers_from_csv(map_obj=m, csv_path="out/kitas_geocoded.csv", color="beige", icon="child", layer_name="Kitas")
#add_markers_from_csv(map_obj=m,csv_path="data/haltestellen_geocoded.csv",color="lightgray", icon="bus", layer_name="Haltestellen")

valid_kita_json = gdf["kitas_route"].apply(lambda x: isinstance(x, str))
print("Gültige JSON-Einträge:", valid_kita_json.sum(), "/", len(gdf))

for _, r in gdf.iterrows():
    c = cluster_colors.get(r.cluster, "#666666")
    folium.CircleMarker(
        location=[r.lat, r.lon],
        radius=3,
        color=c,
        fill=True,
        fill_color=c,
        fill_opacity=0.85,
        tooltip=f"{r.Straßenname} {r.Hsnr} – Cluster {r.cluster}"
    ).add_to(m)

# Optional: Layer Control
folium.LayerControl().add_to(m)

print(gdf.shape)

#m.save("wohnlagen_clusterkarte.html")
m

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Nur Spalten verwenden, die im gdf enthalten sind
cols = [col for col in z_vars if col in gdf.columns]

# Korrelationsmatrix berechnen
corr = gdf[cols].corr()

# Plot
plt.figure(figsize=(20, 16))
sns.heatmap(corr, annot=True, cmap="vlag", center=0)
plt.title("Korrelationsmatrix der Z-Scores")
plt.tight_layout()
plt.show()


Die Korrelationsanalyse zeigt, dass einige Variablen stark korreliert sind, z. B. die verschiedenen Distanzen zu Einzelhandelsstandorten und die Anzahl der Standorte in der Nähe. Dies ist zu erwarten, da Adressen, die näher an Einzelhandelsstandorten liegen, tendenziell auch mehr Standorte in ihrer Umgebung haben. Gleichzeitig ist die Zentralität erwartungskonform leicht mit der Verfügbarkeit von Nahversorgung (Einkaufsmöglichkeiten, medizinische Versorgung, Kitas) korreliert. Diese Erkenntnisse sind Hinweise auf die Plausibilität des Modells, wobei gleichzeitg keine perfekten Korrelationen vorliegen, die auf Redundanzen hindeuten würden. Alle bisher betrachteten Kriterien scheinen relevante und unterschiedliche Aspekte der Wohnlage zu erfassen.

## Interaktive Karte für Bewertung einzelner Adressen

In [ ]:

import json
import ipywidgets as widgets
from IPython.display import display, clear_output

# Widget für Adresseingabe
text_input = widgets.Text(
    value='Immenweg 56',
    placeholder='Straßenname Hausnummer',
    description='Adresse:',
    disabled=False
)

# Button
button = widgets.Button(description="Zeige Routen")

# Ausgabe-Bereich für die Karte
output = widgets.Output()

# Funktion zum Einfügen einer Route + Zielmarker
def add_route(m, geojson_str, color, label, icon, distance=None):
    if isinstance(geojson_str, str):
        try:
            geo = json.loads(geojson_str)
            if geo.get("type") == "LineString":
                coords = [(y, x) for x, y in geo["coordinates"]]
                distance_text = f" – {int(distance)} m" if distance else ""
                tooltip_text = f"{label}{distance_text}"

                folium.PolyLine(
                    locations=coords,
                    color=color,
                    weight=4,
                    opacity=0.9,
                    tooltip=tooltip_text
                ).add_to(m)

                end = coords[-1]
                folium.Marker(
                    location=end,
                    icon=folium.Icon(color=color, icon=icon, prefix="fa"),
                    tooltip=f"Ziel: {label}{distance_text}"
                ).add_to(m)
        except Exception as e:
            print(f"Fehler bei {label}: {e}")

# Funktion zum Anzeigen der Karte
def show_routes():
    with output:
        clear_output()
        addr = text_input.value.strip().lower()

        filtered = gdf[gdf["Adresse_merge"].str.lower().str.contains(addr)]
        if filtered.empty:
            print("Keine Adresse gefunden.")
            return

        row = filtered.iloc[0]
        m = folium.Map(location=[row.lat, row.lon], zoom_start=15, tiles="cartodbpositron")
        folium.Marker(location=[row.lat, row.lon], tooltip="Adresse").add_to(m)

        # ▸ Routen einfügen
        add_route(m, row.get("center_route"), "lightgray", "Weg zum Zentrum", "arrows-to-circle", row.get("center_distance_m"))
        add_route(m, row.get("kitas_route"), "orange", "Nächste Kita", "child", row.get("kitas_min_distance_m"))
        add_route(m, row.get("grundschulen_route"), "green", "Nächste Grundschule", "graduation-cap", row.get("grundschulen_min_distance_m"))
        add_route(m, row.get("haltestellen_route"), "gray", "Nächste Haltestelle", "bus", row.get("haltestellen_min_distance_m"))
        add_route(m, row.get("medzentren_route"), "red", "Nächstes Medizinisches Zentrum", "staff-snake", row.get("medzentren_min_distance_m"))
        add_route(m, row.get("einzelhandel_route"), "blue", "Nächster Einzelhandel", "shop", row.get("einzelhandel_min_distance_m"))
        add_route(m, row.get("gruen_route"), "green", "Nächste Freizeit- und Erholungsfläche", "tree", row.get("gruen_min_distance_m"))
        display(m)

# Button-Event
button.on_click(show_routes)

# UI anzeigen
display(text_input, button, output)

m

# Glättung zur Vermeidung von "Insellagen"
Ein Ziel der Analyse ist die Entwicklung zusammenhängender Gebiete, die einer gemeinsamen Wohnlage zugeordnet werden können. Damit sollen "Insellagen", also mehrere abgeschnittene Bereiche mit derselben Wohnlage vermieden werden. Dafür wenden wir im Folgenden eine Glättung mit dem SKATER-Ansatz an (vgl. [Assunção et al. 2006](https://www.tandfonline.com/doi/abs/10.1080/13658810600665111])).

Standardmäßig wird Queen-Kontinuität für die Nachbarschaftsdefinition verwendet. Da unsere Adressen jedoch als Punkte vorliegen, verwenden wir stattdessen k-Nearest-Neighbors (kNN) mit k=15, um eine ausreichende Vernetzung sicherzustellen. Zusätzlich wird ein Gabriel-Graph über die Punkte gelegt, um die Konnektivität weiter zu verbessern und Cluster auch über größere Distanzen zu ermöglichen (z. B. entlang von Hauptverkehrsachsen) und räumlich korrekte Cluster zu erzeugen.

In [ ]:
from libpysal import weights
from spopt.region import Skater

# Schritt 1: Erzeuge Gewichtsmatrix für SKATER mit Queen-Kontiguität
W = weights.contiguity.Queen.from_dataframe(gdf, use_index=True)

# Schritt 2: Wähle Attribute für SKATER
# ---- SKATER nur mit den Score-Variablen füttern --------
# Keine Routen-Variablen
score_vars = [var for var in score_vars if not var.endswith("_route")]

# Keine Zentrumsdistanz, um konzentrische Cluster zu vermeiden
if "center_distance_m" in score_vars:
    score_vars.remove("center_distance_m")

# Auswahl der Variablen basierend auf Korrelationsanalyse und inhaltlicher Relevanz
attrs = [
    # Lage
    "z_centrality",
    "z_laerm_index_tag",

    # Grün/Wasser
    "z_gruen_distance",
    "z_dist_gross_gruen",
    "z_dist_gross_sonstiges",

    # ÖPNV
    "z_haltestelle_distance",
    "z_headway_score",

    # Bildung
    "z_kita_distance",
    "z_grundschulen_distance",

    # Versorgung
    "z_medzentrum_distance",
    "z_einzelhandel_distance",

    # Industrie (negativ)
    "z_dist_gross_gewerbe",

    # Barriere
    "behind_rail_from_center",

    # Nutzungsklassen (weiche Struktur)
    "nutz_Wohnen",
    "nutz_Gruen",
    "nutz_Gemischt",
    "nutz_Gewerbe",
    "nutz_Verkehr",
]

# Schritt 3: SKATER ausführen
sk = Skater(
    gdf, W, attrs,
    n_clusters=NUMBER_OF_CLUSTERS,
    islands = "increase",
)
sk.solve()

gdf["cluster_skater"] = sk.labels_

In [ ]:
import pandas as pd, folium

m = folium.Map(location=CITY_CENTER, zoom_start=13, tiles="cartodbpositron")
cluster_colors = {
    0:  "#e41a1c",  # kräftiges Rot
    1:  "#377eb8",  # kräftiges Blau
    2:  "#4daf4a",  # kräftiges Grün
    3:  "#984ea3",  # kräftiges Lila
    4:  "#ff7f00",  # Orange
    5:  "#ffff33",  # Gelb
    6:  "#a65628",  # Braun
    7:  "#f781bf",  # Pink
    8:  "#999999",  # Mittelgrau
    9:  "#1b9e77",  # türkis-grün
    10: "#d95f02",  # warmes Orange
    11: "#7570b3",  # Blau-Lila
    12: "#e7298a",  # Magenta
    13: "#66a61e",  # Olivgrün
    14: "#e6ab02",  # Senfgelb
    15: "#a6761d",  # Ocker
    16: "#666666",  # Dunkelgrau
    17: "#1f78b4",  # Dunkelblau
    18: "#b2df8a",  # blasses Grün
    19: "#6a3d9a",  # Dunkellila
}

for _, r in gdf.iterrows():
    c = cluster_colors.get(r.cluster_skater, "#666666")
    folium.CircleMarker(
        location=[r.lat, r.lon],
        radius=3,
        color=c,
        fill=True,
        fill_color=c,
        fill_opacity=0.85,
        tooltip=f"{r.Straßenname} {r.Hsnr} – Cluster {r.cluster_skater}"
    ).add_to(m)

m

# Lageklassen
Abschließend werden die Adressen basierend auf ihrem Gesamt-Score in Lageklassen eingeteilt.

In [ ]:
def classify_lage_5(score_series):
    """
    5 Klassen: A–E, basierend auf Quintilen
    """
    q20 = score_series.quantile(0.20)
    q40 = score_series.quantile(0.40)
    q60 = score_series.quantile(0.60)
    q80 = score_series.quantile(0.80)

    def classify(x):
        if pd.isna(x):
            return None
        if x >= q80:
            return "A – Höchstlage"
        elif x >= q60:
            return "B – Sehr gute Lage"
        elif x >= q40:
            return "C – Gute Lage"
        elif x >= q20:
            return "D – Mittlere Lage"
        else:
            return "E – Einfache Lage"

    return score_series.apply(classify)

# Anwendung:
gdf["lageklasse_5"] = classify_lage_5(gdf["score_total"])

# Farben für Wohnlagen (grün - gelb)
color_map = {
    "A – Höchstlage":             "#1a9850",
    "B – Sehr gute Lage":         "#66bd63",
    "C – Gute Lage":              "#a6d96a",
    "D – Mittlere Lage":          "#d9ef8b",
    "E – Einfache Lage":          "#fee08b",
}

# Kartendarstellung
m = folium.Map(location=CITY_CENTER, zoom_start=13, tiles="cartodbpositron")
for _, r in gdf.iterrows():
    color = color_map.get(r.lageklasse_5, "lightgray")
    folium.CircleMarker(
        location=[r.lat, r.lon],
        radius=3,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.85,
        tooltip=f"{r.Straßenname} {r.Hsnr} – Lageklasse: {r.lageklasse_5}"
    ).add_to(m)
# Optional: Layer Control
folium.LayerControl().add_to(m)
m

# Abgleich mit Ortsteilen, Quartieren und Mietkategorien
Zum Abgleich der ermittelten Wohnlagen mit bestehenden Strukturen werden die Cluster mit den Quartieren und Ortsteilen der Adressen sowie den Mietkategorien (sofern vorhanden) verglichen. In der Kreuztabelle können die Verteilungen der Cluster über die verschiedenen räumlichen Einheiten analysiert werden.

## Ortsteile ergänzen

In [ ]:
import json
import folium
import geopandas as gpd

# Datei laden
with open("data/ortsteile_brandenburg.json", "r", encoding="utf-8") as f:
    raw = json.load(f)

# FeatureCollection extrahieren
features = raw["features"]
gdf_ortsteile = gpd.GeoDataFrame.from_features(features)
gdf_ortsteile.set_crs(EPSG_4326, inplace=True)
gdf_ortsteile = gdf_ortsteile.clip(CITY_BOUNDING_BOX)

# Karte zentrieren
m = folium.Map(location=CITY_CENTER, zoom_start=12, tiles="cartodbpositron")

# Tooltip konfigurieren
tooltip_fields = ["otl_name"]

# GeoJSON mit Tooltip zur Karte hinzufügen
folium.GeoJson(
    gdf_ortsteile,
    name="Ortsteile",
    tooltip=folium.GeoJsonTooltip(
    fields=tooltip_fields,
    aliases=["Ortsteil"],
    localize=True,
    sticky=True
)
).add_to(m)

gdf = gdf.to_crs(EPSG_4326)
gdf_ortsteile = gdf_ortsteile.to_crs(EPSG_4326)

assert gdf.crs.to_epsg() == 4326
assert gdf_ortsteile.crs.to_epsg() == 4326

gdf = clean_index_cols(gdf)
gdf = gpd.sjoin(gdf, gdf_ortsteile[["geometry", "otl_name"]], how="left", predicate="within")
gdf = gdf.rename(columns={"otl_name": "ortsteil"})
gdf = gdf.drop(columns=["index_right"], errors="ignore")

# Drop rows with NaN lat/lon
gdf = gdf.dropna(subset=["lat"])

# Karte anzeigen
m

## Wohnquartiere ergänzen
Innerhalb des Stadtgebiets liegen verschiedene Wohnquartiere mit unterschiedlichen Merkmalen. Diese werden den Adressen zugeordnet und zusammen mit den Wohnbezirken visualisiert.

In [ ]:
import json
import folium
import geopandas as gpd

# Geopackage aus Datei laden
gdf_quartiere = gpd.read_file("data/Quartiere/2024_Quartiere.gpkg")

# Karte zentrieren
m = folium.Map(location=CITY_CENTER, zoom_start=12, tiles="cartodbpositron")

# Ortsteile hinzufügen (s.o.)
tooltip_fields = ["otl_name"]
folium.GeoJson(
    gdf_ortsteile,
    name="Ortsteile",
    tooltip=folium.GeoJsonTooltip(
    fields=tooltip_fields,
    aliases=["Ortsteil"],
    localize=True,
    sticky=True
)
).add_to(m)

# Tooltip konfigurieren
tooltip_fields = ["bezeichnun", "mietkatego"]
tooltip = folium.GeoJsonTooltip(
    fields=tooltip_fields,
    aliases=["Quartier", "Mietkategorie"],
    localize=True,
    sticky=True
)

# GeoJSON mit Tooltip zur Karte hinzufügen als orangefarbener Layer
folium.GeoJson(
    gdf_quartiere,
        style_function=lambda feature: {
            'fillColor': 'orange',
            'color': 'orange',
            'weight': 1,
            'fillOpacity': 0.2,
        },
    name="Ortsteile",
    tooltip=tooltip
).add_to(m)

# CRS angleichen (WGS84)
gdf_quartiere = gdf_quartiere.to_crs(4326)
gdf = gdf.to_crs(4326)
assert gdf.crs.to_epsg() == 4326
assert gdf_quartiere.crs.to_epsg() == 4326

# alte Spalten sicher entfernen
gdf = gdf.drop(columns=["index_right", "mietkatego", "bezeichnun"], errors="ignore")
# Frühere Merge-Spalten entfernen (endet auf _quartier)
cols_to_drop = [col for col in gdf.columns if col.endswith("_quartier")]
gdf = gdf.drop(columns=cols_to_drop, errors="ignore")

# sauberes Spatial Join
gdf = gpd.sjoin(
    gdf,
    gdf_quartiere[["bezeichnun", "mietkatego", "geometry"]],
    how="left",
    predicate="within",
)

# Merkmale benennen
gdf = gdf.rename(columns={
    "bezeichnun": "quartier",
    "mietkatego": "mietkategorie"
})

# Karte anzeigen
m

## Kreuztabelle
Innerhalb des Stadtgebiets liegen verschiedene Wohnquartiere mit unterschiedlichen Merkmalen. Diese werden den Adressen zugeordnet und zusammen mit den Wohnbezirken visualisiert.


In [ ]:
ct = pd.crosstab(gdf["quartier"], gdf["cluster_skater"], normalize="index")
plt.figure(figsize=(14,10))
sns.heatmap(ct, cmap="viridis", annot=False)
plt.title("Quartier vs. SKATER-Cluster (Zeilen normalize: Anteil pro Quartier)")
plt.xlabel("SKATER-Cluster")
plt.ylabel("Quartier")
plt.tight_layout()
plt.show()

Die Darstellung zeigt, wie sich die automatisch gebildeten SKATER-Cluster auf die bestehenden Wohnquartiere verteilen. Deutlich wird, dass mehrere Quartiere eine klare Dominanz einzelner Cluster aufweisen (z. B. Zentrum, Nord, Görden oder Hohenstücken), was auf eine vergleichsweise homogene interne Struktur schließen lässt. Andere Quartiere, insbesondere jene mit gemischten Nutzungen oder großen räumlichen Ausdehnungen, verteilen sich stärker auf mehrere Cluster. Diese Heterogenität ist erwartbar und spiegelt eher die interne Vielfalt der Quartiere wider als Schwächen im Clustering. Insgesamt zeigt die Heatmap, dass die Clusterbildung bestehende räumliche Zusammenhänge gut trifft.

In [ ]:
ct = pd.crosstab(gdf["ortsteil"], gdf["cluster_skater"], normalize="index")

plt.figure(figsize=(12,8))
sns.heatmap(ct, cmap="magma", annot=False)
plt.title("Ortsteil vs. SKATER-Cluster")
plt.xlabel("SKATER-Cluster")
plt.ylabel("Ortsteil")
plt.tight_layout()
plt.show()

Die Zuordnung der SKATER-Cluster zu den Ortsteilen zeigt überwiegend klare Muster. Viele Ortsteile werden überwiegend einem oder zwei Clustern zugeordnet, was auf eine konsistente und funktional nachvollziehbare Lagecharakteristik schließen lässt. Dies ist besonders bei peripheren oder dörflichen Ortsteilen sichtbar, die sich typischerweise durch ähnliche Infrastrukturausstattung und Distanzlagen auszeichnen. Gleichzeitig treten - abhängig von Größe und Struktur des Ortsteils – einzelne Streuungen auf, die auf interne Unterschiede oder Übergangsbereiche hindeuten können. Insgesamt bestätigt die Darstellung, dass die Cluster auch außerhalb des Kernstadtgebiets sinnvolle, zusammenhängende Lagemuster abbilden.

In [ ]:
ct = pd.crosstab(gdf["mietkategorie"], gdf["cluster_skater"], normalize="columns")

plt.figure(figsize=(10,6))
sns.heatmap(ct, cmap="coolwarm", annot=True, fmt=".2f")
plt.title("Mietkategorie vs. SKATER-Cluster (Spalten normalize)")
plt.xlabel("SKATER-Cluster")
plt.ylabel("Mietkategorie")
plt.tight_layout()
plt.show()


Die Kreuztabelle zwischen Mietkategorien und SKATER-Clustern dient der ökonomischen Validierung des Modells. Hier zeigt sich, dass bestimmte Cluster deutlich mit niedrigeren, mittleren oder höheren Mietkategorien assoziiert sind. Mehrere Cluster weisen eine hohe Übereinstimmung mit spezifischen Mietniveaus auf, was darauf hinweist, dass die algorithmisch erkannten Lagegruppen auch sozioökonomische Unterschiede in der Wohnlagequalität widerspiegeln. Streuungen in einzelnen Kategorien sind zu erwarten und spiegeln natürliche Übergänge oder heterogene Straßenzüge wider. Insgesamt deutet die Struktur jedoch auf eine plausibel differenzierende Wirkung der Cluster hin.

In [ ]:
# Alle relevanten Z-Variablen für die Clusterinterpretation
z_vars = [col for col in gdf.columns if col.startswith("z_")]

cluster_profile = gdf.groupby("cluster_skater")[z_vars].mean()

plt.figure(figsize=(18,10))
sns.heatmap(cluster_profile, cmap="coolwarm", center=0, annot=False)
plt.title("Feature-Profil der SKATER-Cluster (alle Z-Scores)")
plt.xlabel("Feature")
plt.ylabel("Cluster")
plt.tight_layout()
plt.show()


# Export-Pipeline

In [ ]:
# Save GeoDataFrame with scores and clusters as CSV
import os

# -------------------------------------------
# 0) EXPORT-PFAD
# -------------------------------------------
EXPORT_PATH = "out"
EXPORT_FILE = os.path.join(EXPORT_PATH, "wohnlagen_brb.gpkg")

# Ordner anlegen
os.makedirs(EXPORT_PATH, exist_ok=True)

# Falls alte Datei existiert: löschen (sonst doppelte Layer)
if os.path.exists(EXPORT_FILE):
    os.remove(EXPORT_FILE)
    print(f"Alte Datei gelöscht: {EXPORT_FILE}")

# -------------------------------------------
# 1) HILFSFUNKTION: Sicheres Schreiben
# -------------------------------------------

def write_layer(gdf, layer_name, crs=EPSG_4326):
    """
    Schreibt einen GeoDataFrame als Layer in die GeoPackage-Datei.
    Stellt sicher, dass das CRS korrekt ist.
    """
    if gdf is None or len(gdf) == 0:
        print(f"⚠ Layer '{layer_name}' übersprungen: leer oder None")
        return

    # CRS prüfen
    if gdf.crs is None:
        print(f"⚠ GDF '{layer_name}' hat kein CRS – setze auf {crs}")
        gdf = gdf.set_crs(crs)
    elif gdf.crs.to_string() != crs:
        print(f"🔄 Reprojiziere '{layer_name}' nach {crs}")
        gdf = gdf.to_crs(crs)

    # Schreiben
    gdf.to_file(EXPORT_FILE, layer=layer_name, driver="GPKG")
    print(f"✔ Exportiert: {layer_name}  →  {EXPORT_FILE}")


# -------------------------------------------
# 2) LAYER DEFINIEREN
# -------------------------------------------
# Du kannst hier ALLE relevanten Datensätze aufnehmen:

layers = {
    "wohnadressen": gdf,
    "wohnlagen_score": gdf[[col for col in gdf.columns if col.startswith("score_") or col in ["geometry"]]],
    "cluster_skater": gdf[["cluster_skater", "geometry"]] if "cluster_skater" in gdf.columns else None,
    "cluster_kmeans": gdf[["cluster", "geometry"]] if "cluster" in gdf.columns else None,
    "lageklassen": gdf[["lageklasse", "score_total", "geometry"]] if "lageklasse" in gdf.columns else None,

    # Infrastruktur-Daten
    "bahnlinien": rails,
    "gruenflaechen": gdf_gruen_shape,
    #"laerm_lden": gdf_laerm,                      # falls du gdf_laerm heißt
    "haltestellen": gdf_haltestellen if 'gdf_haltestellen' in globals() else None,
    #"medzentren": gdf_medzentren if 'gdf_medzentren' in globals() else None,
    "kitas": gdf_kitas if 'gdf_kitas' in globals() else None,
    "grundschulen": gdf_grundschulen if 'gdf_grundschulen' in globals() else None,
    #"einzelhandel": gdf_einzelhandel if 'gdf_einzelhandel' in globals() else None,

    # Debug-Daten
    "bahnschnitt_debug": gdf[["behind_rail_from_center", "center_route", "geometry"]] if "behind_rail_from_center" in gdf.columns else None,
}

# -------------------------------------------
# 3) EXPORT AUSFÜHREN
# -------------------------------------------

print("🔁 Exportiere alle Layer …\n")

for layer_name, layer_gdf in layers.items():
    write_layer(layer_gdf, layer_name)

print("\n🎉 Fertig! Die GeoPackage-Datei liegt hier:")
print(f"📦 {EXPORT_FILE}")
